# Multi-Omics Imputation

The plan is to do the following:
- Divide the data into train, validation, and test sets. Leave the test set for later.
- Remove the values for a certain omics type from the validation set
- Impute using the train set and the remaining value in the validation set
- Compare these imputed values against the true values
    - Distribution of correlation coefficients
    - Get the mean and stdev of the correlation coefficients
    - Choose best model
- Evaluate on test set
- Choose best method
- Try on independent set
- Finally, train GCN model and see difference between single omics, multi-omics, and imputed multi-omics

I'll first start with some basics: data import and processing.
Then I'll move to imputing one omics from two.
Then I'll move to imputing two omics from one.
I'll do all the steps above along the way.

## Importing requisite packages

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from metrics import *

## Importing data

In [17]:
mrna = pd.read_csv("../R/TCGA BRCA/mrna_top1000.csv", index_col=0)
meth = pd.read_csv("../R/TCGA BRCA/meth_top1000.csv", index_col=0)
mirna = pd.read_csv("../R/TCGA BRCA/mirna_anova.csv", index_col=0)

labels = pd.read_csv("../R/TCGA BRCA/PAM50_subtype.csv", index_col=0)

## Basic Data Processing

Just combining all data and then also having a list containing what datatype the columns belong to.

In [18]:
all_data = pd.merge(pd.merge(mrna, meth, left_index=True, right_index=True), mirna,  left_index=True, right_index=True)

datatypes = ["mrna"]*mrna.shape[1] + ["meth"]*meth.shape[1] + ["mirna"]*mirna.shape[1]

In [19]:
all_data = (all_data-all_data.min())/(all_data.max() - all_data.min())
all_data.head()

,DBF4|10926,DACH1|1602,BBS4|585,L3MBTL4|91133,TK1|7083,KIAA1370|56204,GPD1L|23171,RERG|85004,RAPGEF3|10411,FBXO36|130888,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A1XU-01,-1.161262,0.901339,-0.526396,-0.680798,-0.877519,-0.244543,-0.328603,1.344207,0.282989,0.821016,...,-0.099680,0.057183,1.178557,-0.050144,-0.200428,0.312028,0.258953,1.400702,-0.093096,0.690934
TCGA-D8-A1XV-01,0.211779,1.266096,1.625856,1.083625,-0.015763,1.745320,0.398140,1.242530,-0.463320,1.234436,...,-0.125443,-0.678719,-0.063220,-0.109349,-0.180856,0.763328,0.138603,-0.724265,-0.222224,-0.004114
TCGA-E9-A1N3-01,0.191226,2.116788,0.864832,-1.814764,0.414139,0.442796,-0.302383,1.190065,0.047615,1.754033,...,-0.119718,-0.319499,3.662111,-0.097890,-0.180856,0.382248,0.299069,-0.183365,0.692901,1.417576
TCGA-C8-A1HE-01,-0.668288,1.509150,1.250869,-1.087240,-0.188361,1.132530,1.699859,0.988028,0.642218,1.533607,...,-0.154070,-0.172319,-0.063220,-0.088340,-0.131925,0.431559,-0.342798,-0.685630,-0.255909,0.217038
TCGA-A1-A0SQ-01,-0.274364,0.574516,0.483963,-1.037509,0.255192,0.198792,0.456748,0.850052,-0.177073,0.244715,...,-0.168383,-0.758546,-0.477146,-0.115078,-0.161283,-0.660002,-0.382915,-0.994716,-0.272752,2.238997


In [20]:
labels.head()

,cancer_subtype
patient_id,
TCGA-D8-A1XU-01,LumA
TCGA-D8-A1XV-01,LumA
TCGA-E9-A1N3-01,LumA
TCGA-C8-A1HE-01,LumA
TCGA-A1-A0SQ-01,LumA


Doing the train-validation-test split.
These contain all the values intact.  

Here, we do a 60-20-20 split.

In [21]:
X_train, X_test, y_train, y_test = train_test_split(all_data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.25, random_state = 42, stratify = y_train)


print(X_train.shape)
print(X_val.shape)
print(X_test.shape)
print(y_train.shape)
print(y_val.shape)
print(y_test.shape)

#all_data.head()
#labels.head()
#X_train.head()
#y_train.head()

(372, 2257)
(125, 2257)
(125, 2257)
(372, 1)
(125, 1)
(125, 1)


Removing all miRNA feature values

In [22]:
#Keeping values for later
from copy import deepcopy
X_test_truth = deepcopy(X_test)
X_val_truth = deepcopy(X_val)

mask = [x=="mirna" for x in datatypes]
X_test.loc[:,mask] = np.nan
X_val.loc[:,mask] = np.nan

X_test.loc[:,mask].head()

/home/bs16b001/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/bs16b001/.local/lib/python3.7/site-packages/pandas/core/indexing.py:691: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


,hsa-mir-576,hsa-mir-200b,hsa-mir-3687,hsa-mir-126,hsa-mir-26a-2,hsa-mir-101-1,hsa-mir-218-2,hsa-mir-223,hsa-mir-335,hsa-mir-1468,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-A2-A3XX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A0DI-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-A7-A6VX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A0AZ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-AR-A1AX-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Simple Imputation Methods

In [23]:
from sklearn.impute import SimpleImputer

#Combining the train and test samples into one dataframe.
print(X_train.shape)
print(X_val.shape)
X = pd.concat([X_train, X_val])
print(X.shape)
X.iloc[370:375, mask]

(372, 2257)
(125, 2257)
(497, 2257)


,hsa-mir-576,hsa-mir-200b,hsa-mir-3687,hsa-mir-126,hsa-mir-26a-2,hsa-mir-101-1,hsa-mir-218-2,hsa-mir-223,hsa-mir-335,hsa-mir-1468,...,hsa-mir-217,hsa-mir-424,hsa-mir-581,hsa-mir-483,hsa-mir-3614,hsa-mir-16-1,hsa-mir-550a-2,hsa-mir-24-1,hsa-mir-508,hsa-mir-642a
patient_id,,,,,,,,,,,,,,,,,,,,,
TCGA-D8-A1JB-01,-0.196327,-0.469305,-0.379658,0.335229,-0.200882,-0.056515,-0.107554,-0.248959,-0.240506,-0.287123,...,-0.171246,0.312878,-0.891071,0.049167,-0.190642,-0.430013,0.058369,-0.550404,-0.171695,-0.509604
TCGA-D8-A1JL-01,0.115839,0.515563,-0.257932,-0.126773,0.112686,-0.689612,0.067629,-0.057760,0.293802,-0.304254,...,2.197596,1.435441,2.834259,-0.111258,0.249737,0.772796,0.058369,1.207523,-0.216609,-0.446418
TCGA-EW-A2FS-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-BH-A8FZ-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-D8-A1XG-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Imputing with Mean and Median

### Imputing with Mean

In [9]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_train)

mean_imputed = imp.transform(X_val)
# SimpleImputers returns a numpy.ndarray
# I will convert it to a pandas data frame
#mean_imputed = pd.DataFrame(mean_imputed, columns = X_val.columns, index = X_val.index)


#print(mean_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#mean_imputed.loc[:,mask].head()

### Imputing with Median

In [10]:
imp = SimpleImputer(missing_values=np.nan, strategy='median')
imp.fit(X_train)

median_imputed = imp.transform(X_val)
#median_imputed = pd.DataFrame(median_imputed, columns = X_val.columns, index = X_val.index)

#print(median_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#median_imputed.loc[:,mask].head()

In [11]:
mask = [x=="mirna" for x in datatypes]
truth = X_val_truth.loc[:,mask].to_numpy()
random = (np.random.rand(truth.shape[0],truth.shape[1]))# - np.mean(truth))/np.std(truth)

print("RMSE")
print(rmse(truth, truth))
print(rmse(truth, random))
print(rmse(truth, mean_imputed[:,mask]))
print(rmse(truth, median_imputed[:,mask]))

print("\nStandard Deviation")
print(truth.std())
print(random.std())
print(mean_imputed[:,mask].std())
print(median_imputed[:,mask].std())

RMSE
0.0
0.5274265431218255
0.09808654721684143
0.10037655901358895

Standard Deviation
0.10615528918894765
0.2895128760819338
0.04489987012416516
0.03600215901869758


# Slightly More Complicated Methods

In [12]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

Here, initially, all the missing features are replaced with the mean value. Then, iteratively, the estimator is used to estimate the missing value from all the other features. Within each iteration through all the missing values, the features are imputed in a random order.

## Estimator: ElasticNet

#### L1 Ratio = 0.2

In [13]:
from sklearn.linear_model import ElasticNet

imp = IterativeImputer(estimator = ElasticNet(l1_ratio = 0.2), initial_strategy = "mean", 
                       imputation_order = "random", random_state = 42,
                      n_nearest_features = 50)
imp.fit(X_train)

elasticnet_l2_imputed = imp.transform(X_val)
#elasticnet_l2_imputed = pd.DataFrame(elasticnet_l2_imputed, columns = X_val.columns, index = X_val.index)
#print(elasticnet_l2_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#elasticnet_l2_imputed.loc[:,mask].head()

#### L1 Ratio = 0.8

In [14]:
from sklearn.linear_model import ElasticNet

imp = IterativeImputer(estimator = ElasticNet(l1_ratio = 0.75), initial_strategy = "mean", 
                       imputation_order = "random", random_state = 42,
                      n_nearest_features = 50)
imp.fit(X_train)

elasticnet_l1_imputed = imp.transform(X_val)
#elasticnet_l1_imputed = pd.DataFrame(elasticnet_l1_imputed, columns = X_val.columns, index = X_val.index)
#print(elasticnet_l1_imputed.shape)
#mask = [x=="mirna" for x in datatypes]
#elasticnet_l1_imputed.loc[:,mask].head()

In [15]:
print("RMSE")
print(rmse(truth, truth))
print(rmse(truth, mean_imputed[:,mask]))
print(rmse(truth, elasticnet_l1_imputed[:,mask]))
print(rmse(truth, elasticnet_l2_imputed[:,mask]))

print("\nStandard Deviation")
print(truth.std())
print(mean_imputed[:,mask].std())
print(elasticnet_l1_imputed[:,mask].std())
print(elasticnet_l2_imputed[:,mask].std())

RMSE
0.0
0.09808654721684143
0.09808654721684143
0.09808654721684143

Standard Deviation
0.10615528918894765
0.04489987012416516
0.04489987012416516
0.04489987012416516


## Estimator: KNeighborsRegressor

#### K = 25

In [42]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=25), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 25)
imp.fit(X_train)

knn25_iter_imputed = imp.transform(X_val)

#### K = 50

In [43]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=50), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 50)
imp.fit(X_train)

knn50_iter_imputed = imp.transform(X_val)

#### K = 75

In [44]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=75), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 75)
imp.fit(X_train)

knn75_iter_imputed = imp.transform(X_val)

#### K = 100

In [45]:
from sklearn.neighbors import KNeighborsRegressor

imp = IterativeImputer(estimator = KNeighborsRegressor(n_neighbors=100), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 100)
imp.fit(X_train)

knn100_iter_imputed = imp.transform(X_val)

In [46]:
print("RMSE")
print(rmse(truth, truth))
print(rmse(truth, mean_imputed[:,mask]))
print(rmse(truth, knn25_iter_imputed[:,mask]))
print(rmse(truth, knn50_iter_imputed[:,mask]))
print(rmse(truth, knn75_iter_imputed[:,mask]))
print(rmse(truth, knn100_iter_imputed[:,mask]))

print("\nStandard Deviation")
print(truth.std())
print(mean_imputed[:,mask].std())
print(knn25_iter_imputed[:,mask].std())
print(knn50_iter_imputed[:,mask].std())
print(knn75_iter_imputed[:,mask].std())
print(knn100_iter_imputed[:,mask].std())

RMSE
0.0
0.09808654721684143
0.09706443121270594
0.09684563410179363
0.09688083879202193
0.09683528846469257

Standard Deviation
0.10615528918894765
0.04489987012416516
0.04442395037412617
0.04136561038910045
0.04102801872375675
0.040607221785942914


## Estimator: RandomForestRegressor

#### Max Depth = 10

In [11]:
from sklearn.ensemble import RandomForestRegressor

imp = IterativeImputer(estimator = RandomForestRegressor(max_depth=10), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 200)
imp.fit(X_train)

rf10_imputed = imp.transform(X_val)

#### Max Depth = 20

In [12]:
from sklearn.ensemble import RandomForestRegressor

imp = IterativeImputer(estimator = RandomForestRegressor(max_depth=20), 
                       initial_strategy = "mean", imputation_order = "random", random_state = 42,
                      n_nearest_features = 200)
imp.fit(X_train)

rf20_imputed = imp.transform(X_val)

In [13]:
print(nrmse(truth, mean_imputed[:,mask]))
print(nrmse(truth, rf10_imputed[:,mask]))
print(nrmse(truth, rf20_imputed[:,mask]))

1.3514235544024777
1.3526947398648599
1.353721124349755


# Deck Imputation

It is not exactly deck imputation in that we are not replacing the missing value with a value from the existing set. Here, I select the k closest samples and get the average of their values to impute the missing values.

I am testing kNN with k values equal to all odd numbers between 0 and 20.

In [16]:
from sklearn.impute import KNNImputer

knn = {}

for i in [1,5,10,15,20,25,30,35,40,45,50,75,100]:
    imputer = KNNImputer(n_neighbors=i)
    imputer.fit(X_train)

    knn[i] = imputer.transform(X_val)
    #knn[i] = pd.DataFrame(knn[i], columns = X_val.columns, index = X_val.index)

print(len(knn))
#print(knn[1].shape)
#mask = [x=="mirna" for x in datatypes]
#knn[1].loc[:,mask].head()

13


In [20]:
print(rmse(truth, truth))
print(truth.std())
print()
print(rmse(truth, mean_imputed[:,mask]))
print(mean_imputed[:,mask].std())
print()

print(min([rmse(truth, each[:,mask]) for each in knn.values()]))
print(max([each[:,mask].std() for each in knn.values()]))
for each in knn.values():
    print()
    print(rmse(truth, each[:,mask]))
    print(each[:,mask].std())

0.0
0.10615528918894765

0.09808654721684143
0.04489987012416516

0.09772186476154054
0.10478754862122502

0.13112292928554084
0.10478754862122502

0.1047475141839711
0.07203862173704595

0.10097084589363316
0.06427839857282282

0.09977679942888053
0.06140562119523913

0.09925989347980214
0.059305083427441035

0.09856044397313468
0.05781558990162404

0.0980691452730506
0.05642601332976928

0.09803534611140967
0.055640456876138306

0.09816291728792632
0.054956696204829354

0.09784663394111433
0.0542942718341059

0.09772186476154054
0.05382403291660356

0.0977562975188772
0.05158934607844852

0.09773882456164201
0.050021110608876346


In [18]:
?max

# Comparing Imputation Methods

To compare the imputation methods, we first need to quantify them. Here, I am going to use the Normalized Root Mean Squared Error (NRMSE) to quantify each of the methods and then compare them.

< Insert NRMSE formula in latex >

## Truth and Random values' RMSE and Standard Deviation

In [24]:
missing_types = [["mirna"], ["meth"], ["mrna"], ["mirna", "meth"], ["meth", "mrna"], ["mrna", "mirna"]]
df_rows_rmse = []
df_rows_std = []
for missing in missing_types:
    print("\n", "="*50)
    print("Missing datatype = ", missing, "\n")
    
    mask = [x in missing for x in datatypes]
    X_test = deepcopy(X_test_truth)
    X_test.loc[:,mask] = np.nan
    truth = X_test_truth.loc[:,mask].to_numpy()
    random = (np.random.rand(truth.shape[0],truth.shape[1]))
    
    imp = SimpleImputer(missing_values=np.nan, strategy="mean")
    imp.fit(X_train)
    mean = imp.transform(X_test)

    print("RMSE")
    print(rmse(truth, truth))
    print(rmse(truth, random))
    print(rmse(truth, mean[:,mask]))

    print("\nStandard Deviation")
    print(truth.std())
    print(random.std())
    print(mean[:,mask].std())


Missing datatype =  ['mirna'] 

RMSE
0.0
1.0443165454585634
0.8383247718459718

Standard Deviation
0.8319837004150066
0.28783779755410227
0.025670379034524585

Missing datatype =  ['meth'] 

RMSE
0.0
1.0998469585973274
0.9300887498403106

Standard Deviation
0.927704455096941
0.2887190241646182
0.02664959907034445

Missing datatype =  ['mrna'] 

RMSE
0.0
1.1268310908874701
0.9691842723851896

Standard Deviation
0.967248526265358
0.2881864855011066
0.02847088124146084

Missing datatype =  ['mirna', 'meth'] 

RMSE
0.0
1.0887784103542195
0.9120782859888247

Standard Deviation
0.9091377548851294
0.2880835948808805
0.026696378882434788

Missing datatype =  ['meth', 'mrna'] 

RMSE
0.0
1.1136532604368892
0.9498376799270287

Standard Deviation
0.9477228838075595
0.288739979303564
0.029656316830353645

Missing datatype =  ['mrna', 'mirna'] 

RMSE
0.0
1.110481392363853
0.9439059569192432

Standard Deviation
0.9415155350435739
0.28894561186553896
0.030552497670991993


In [15]:
?rmse